# Determination of Optimal Initial Threshold

## I. INTRODUCTION
            
Using lesion probability maps created from LST’s Lesion Growth Algorithm (LGA) helps to automate the process of mapping out the lesions visible in our MS subjects’ MRI Images. Unfortunately these lesion probability maps require time-consuming manual edits to correct the False Positives and False Negative. To minimize this issue we created a script to run LST multiple times while iterating through all possible parameters, bin threshold and initial threshold, and created a new metric to evaluate which parameters create the most accurate lesion probability map for any given FLAIR-T1 image pair.

## II. METHODS
 
### A. 
Our first step in this experiment required writing a script to run LST’s provided tool for determining the optimum threshold on a given lesion probability map, while iterating through its one parameter, binary threshold while keeping initial threshold the same. (0.3) The scripts are displayed below:

In [ ]:
from nipype.interfaces.spm.base import SPMCommand, SPMCommandInputSpec
from nipype.interfaces.base import (BaseInterface, TraitedSpec, traits, File,
                                    OutputMultiPath, BaseInterfaceInputSpec,
                                    isdefined, InputMultiPath)
import nipype.pipeline.engine as pe
import nipype.interfaces.io as nio
from nipype.interfaces.utility import IdentityInterface
import os
#from utils import doit_workflow


class DoitInputSpec(SPMCommandInputSpec):
    data_ref = traits.List(File(exists=True), field="doit.data_ref")
    bin_thresh = traits.Any(0.5, field="doit.bin_thresh", usedefault=True) #default is 0.5


class DoitOutputSpec(TraitedSpec):
    csv_file = OutputMultiPath(File(exists=True))


class Doit(SPMCommand):
    input_spec = DoitInputSpec
    output_spec = DoitOutputSpec
    _jobtype = 'tools'
    _jobname = 'LST'

    def _make_matlab_command(self, contents, postscript=None):
        len_ref = len(self.inputs.data_ref)

        contents = """
        %% Generated by nipype.interfaces.spm
        if isempty(which('spm')),
             throw(MException('SPMCheck:NotFound', 'SPM not in matlab path'));
        end
        [name, version] = spm('ver');
        fprintf('SPM version: %s Release: %s',name, version);
        fprintf('SPM path: %s', which('spm'));
        spm('Defaults','fMRI');

        if strcmp(name, 'SPM8') || strcmp(name(1:5), 'SPM12'),
           spm_jobman('initcfg');
           spm_get_defaults('cmdline', 1);
        end
        """

        contents += """
        jobs{1}.spm.tools.LST.doit.bin_thresh = %f;
        """ % self.inputs.bin_thresh

        for i, ref in enumerate(self.inputs.data_ref):
            contents += """
            jobs{1}.spm.tools.LST.doit.data_ref{%d, 1} = '%s';
        """ % (i+1,
               ref)

        contents += """
                    spm_jobman('run', jobs);
                    """
        return contents

    def _format_arg(self, opt, spec, val):
        """Convert input to appropriate format for spm
        """
        # import numpy as np
        # from nipype.utils.filemanip import copyfiles

        # if opt in ['t1_files', 'flair_files']:
        #    val2 = copyfiles(val, os.path.abspath("."))
        #    return np.array(val2, dtype=object)
        print("_format_arg opt is: ", opt)
        print("_format_arg opt is: ", spec)
        print("_format_arg opt is: ", val)

        # TODO change the format of data_ref

        return super(Doit, self)._format_arg(opt, spec, val)


    def _list_outputs(self):
        #from nipype.utils.filemanip import fname_presuffix
        from glob import glob
        from os.path import join
        outputs = self._outputs().get()
        print("Listing outputs!")
        print(os.path.abspath('.'))

        outputs["csv_file"] = glob(join(os.path.abspath('.'), "LST_doit_*.csv"))
        print(outputs)
        return outputs

__Three python files were written in the same directory. The script above wraps SPM command into python to run LGA. The file name is doit_spm.py__

In [ ]:
__author__ = 'sf713420'

import nipype.pipeline.engine as pe
import nipype.interfaces.io as nio
from nipype.interfaces.utility import IdentityInterface
import os
import sys
print(__file__)
print(sys.path)
sys.path.append(os.path.dirname(__file__))
import numpy as np
from doit_spm import Doit

def flatten(l):
    return [item for sublist in l for item in sublist]

def doit_workflow(data_ref, bin_thresh, base_dir = None, sink_dir = None):
    # data_ref should be a list of pathname
    # base_dir is the working directory
    # sink_dir is where the data is sinking
    if base_dir is None:
        base_dir = os.getcwd()
        print("base_dir is: ", base_dir)
    if sink_dir is None:
        sink_dir = base_dir
        print("sink_dir is: ", sink_dir)

    count = 0


    # inputs
    inputspec = pe.Node(IdentityInterface(fields = ['data_ref', 'thresh']), name = 'inputspec')
    inputspec.inputs.mandatory_inputs = True
    inputspec.inputs.data_ref = data_ref
    inputspec.inputs.thresh = bin_thresh


    # doit_node
    doit_node = pe.Node(name = 'doit_node',
                           interface = Doit(),
                           #iterfield = ['data_ref']
                           )
    doit_node.iterables = ("bin_thresh", bin_thresh)
    # TODO
    print(doit_node.iterables)
    print(doit_node.inputs.bin_thresh)

    #datasink
    data_sink = pe.Node(nio.DataSink(), name = 'sinker')
    data_sink.inputs.base_directory = sink_dir
    data_sink.inputs.container = '.'

    # Pipeline assembly
    pipeline = pe.Workflow(name = 'pipeline_doit')
    pipeline.base_dir = base_dir

    pipeline.connect(inputspec, 'data_ref', doit_node, 'data_ref')
    pipeline.connect(inputspec, 'thresh', doit_node, 'bin_thresh')
    pipeline.connect(doit_node, 'csv_file', data_sink, '@LST_doit')

    pipeline.write_graph(graph2use = 'orig')
    pipeline.config['Execution'] = {'keep_inputs': True, 'remove_unnecessary_outputs': False}

    return pipeline

__A nipype workflow was made to iterate through binary threshold with increment 0.05. The file name is utils.py__

In [ ]:
__author__ = 'sf713420'

import os
from nipype.interfaces.base import isdefined
from utils import doit_workflow, flatten
import sys
from glob import glob
import numpy as np
import csv

if __name__ == '__main__':
    print("Reading file from the path: ", sys.argv[1], "\n")
    data_ref = glob(sys.argv[1])

    FLAIR_T1_name = data_ref[0].split('/')[5]
    output_dir = '/data/henry1/tristan/LST/opt_thresh_results'
    sk_dir = os.path.join(output_dir,
                          FLAIR_T1_name
                          )

    dwf = doit_workflow(data_ref, thresh_array, sink_dir=sk_dir)
    dwf.run()
    
    """
    with open(os.path.join(sk_dir, '_bin_thresh_' + thresh_array[i]), newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')

    csvfile.close()
    """

__This script takes the first positional argument as filename for running the LGA DC calculation workflow. The script is name as doit_arg.py__

The csv files are generated for each binary threshold. All of them includes 102 mses which are FLAIR-MPRAGE image pairs. The average Dice Coefficient (DC), Sensitivity (SE) and Specificity (SP) among these mses were calculated.

### B. 
The next step a script that would run LST’s LGA on a given FLAIR-T1 image pair while iterating through its one parameter, initial threshold was run. The multiple lesion probability maps created would then be analyzed using LST’s provided tool for determining the optimum threshold to evaluate which initial threshold will create a map that requires the least amount of manual edits. The LGA iteration for initial threshold (kappa) increment of 0.05 was run in PBR. 5 test mses (mse3727, mse4413, mse4482, mse4739, mse4754) were run and outputs were calculated using previous code with binary threshold set to 0.3. The code is shown below:

In [ ]:
__author__ = 'sf713420'
from nipype.interfaces.base import (BaseInterface, TraitedSpec, traits, File,
                                    OutputMultiPath, BaseInterfaceInputSpec,
                                    isdefined, InputMultiPath)

from ...config import config
from glob import glob
import os
from ...base import register_workflow, PBRBaseInputSpec, PBRBaseInterface
from nipype.interfaces.spm.base import SPMCommand, SPMCommandInputSpec

class LGAInputSpec(SPMCommandInputSpec):
    t1_files = traits.List(File(exists=True), field="lga.data_T1")
    flair_files = traits.List(File(exists=True), field="lga.data_F2")
    kappa = traits.Float(0.3, field="lga.opts_lga.initial", usedefault=True)
    maxiter = traits.Int(50, field="lga.opts_lga.maxiter", usedefault=True)
    phi = traits.Float(1.0, field="lga.opts_lga.mrf", usedefault=True)
    html_report = traits.Bool(0, field="lga.html_report", usedefault=True)

class LGAOutputSpec(TraitedSpec):
    lesion_probability_map = OutputMultiPath(File(exists=True))
    mat_file = OutputMultiPath(File(exists=True))
    bias_corrected_flair = OutputMultiPath(File(exists=True))


class LGA(SPMCommand):
    input_spec = LGAInputSpec
    output_spec = LGAOutputSpec
    _jobtype = 'tools'
    _jobname = 'LST'

    def _format_arg(self, opt, spec, val):
        """Convert input to appropriate format for spm
        """
        import numpy as np
        from nipype.utils.filemanip import copyfiles

        if opt in ['t1_files', 'flair_files']:
            val2 = copyfiles(val, os.path.abspath("."))
            return np.array(val2, dtype=object)

        return super(LGA, self)._format_arg(opt, spec, val)

    def _list_outputs(self):
        from nipype.utils.filemanip import fname_presuffix
        outputs = self._outputs().get()
        kappa = round(self.inputs.kappa, 2)

        if str(kappa) == '1.0':
            kappa = 1

        outputs["lesion_probability_map"] = [fname_presuffix(f,
                                                prefix="ples_lga_{}_rm".format(kappa),
                                                newpath=os.path.abspath(".")) for f in self.inputs.flair_files]
        outputs["bias_corrected_flair"] = [fname_presuffix(f,
                                                prefix="rm",
                                                newpath=os.path.abspath(".")) for f in self.inputs.flair_files]
        outputs["mat_file"] = [fname_presuffix(f,
                                                prefix="LST_lga_rm",
                                                newpath=os.path.abspath("."),
                                                use_ext=False)+".mat"  for f in self.inputs.flair_files]
        return outputs


def mapper(Nt1, t1):
    return [t1[Nt1]]

def mapper2(Nt2, t2):
    return [t2[Nt2]]


class LGAIterInputSpec(PBRBaseInputSpec):
    t1_files = InputMultiPath(File(exists=True))
    flair_files = InputMultiPath(File(exists=True))

class LGAIterOutputSpec(TraitedSpec):
    lesion_probability_mask_lga = OutputMultiPath(File(exists=True))
    lesion_binary_ref = OutputMultiPath(File(exists=True))
    lga_mat = OutputMultiPath(File(exists=True))
    lga_txt = OutputMultiPath(File(exists=True))
    lga_index = OutputMultiPath(File(exists=True))
    lga_metrics = OutputMultiPath(traits.Dict())

class LGAIter(PBRBaseInterface):
    """
    """
    input_spec = LGAIterInputSpec
    output_spec = LGAIterOutputSpec
    flag = "lga" #this is for pbr mse# -w interfacename
    connections = [("align", "t1_files", "t1_files"),
                   ("align", "flair_files", "flair_files")]

    def _run_interface_pbr(self, runtime):
        from nipype.pipeline.engine import Node, Workflow, MapNode
        from nipype.interfaces.io import DataSink
        import nipype.algorithms.misc as misc
        from nipype.interfaces.utility import IdentityInterface, Function
        import numpy as np

        wf = Workflow(name="lga_%s"%self.inputs.mseID)
        wf.base_dir = os.path.join(config["working_directory"])

        #get_ratio_workflow(config)
        inputspec = Node(IdentityInterface(["t1s", "flairs"]), name="inputspec")
        #wf.get_node("inputspec")
        #inputspec.inputs.examID = self.inputs.mseID

        map_func = Node(Function(input_names=["Nt1", "t1"],
                                 output_names=["t1"],
                                 function=mapper),
                        name="stupid_mapper")

        map_func2 = Node(Function(input_names=["Nt2", "t2"],
                                 output_names=["t2"],
                                 function=mapper2),
                        name="stupid_mapper2")

        map_func.inputs.t1 = self.inputs.t1_files
        map_func2.inputs.t2 = self.inputs.flair_files

        map_func.iterables = [('Nt1', range(len(self.inputs.t1_files)))]
        map_func2.iterables = [('Nt2', range(len(self.inputs.flair_files)))]

        wf.connect(map_func, "t1", inputspec, "t1s")
        wf.connect(map_func2, "t2", inputspec, "flairs")

        gunzip_t1 = MapNode(interface=misc.Gunzip(), name='gunzip_t1', iterfield=["in_file"])
        gunzip_flair = MapNode(interface=misc.Gunzip(), name='gunzip_flair', iterfield=["in_file"])
        wf.connect(inputspec, "t1s", gunzip_t1, "in_file")
        wf.connect(inputspec, "flairs", gunzip_flair, "in_file")

        lga = Node(LGA(), name="lga")
        init_thresh = np.linspace(0.05, 1.00, num=20)
        lga.iterables = ("kappa", init_thresh)
        # or lga.inputs.kappa = init_thresh (?)
        wf.connect(gunzip_t1, "out_file", lga, "t1_files")
        wf.connect(gunzip_flair, "out_file", lga, "flair_files")

        sinker = Node(DataSink(), name="sinker")
        sinker.inputs.base_directory = config["output_directory"]
        sinker.inputs.container = self.inputs.mseID

        import nipype.interfaces.fsl as fsl
        cluster_lga = MapNode(fsl.Cluster(threshold=0.0001,
                                          out_index_file = True,
                                          #out_localmax_txt_file=True,
                                          use_mm=True),
                              name="cluster_lga",
                              iterfield=["in_file"])
        wf.connect(lga, "lesion_probability_map", cluster_lga, "in_file")
        wf.connect(cluster_lga, "index_file", sinker, "lst.lga.cluster")

        from nipype.interfaces.freesurfer import SegStats
        segstats_lga = MapNode(SegStats(), name="segstats_lga", iterfield=["segmentation_file"])
        wf.connect(cluster_lga, "index_file", segstats_lga, "segmentation_file")

        wf.connect(segstats_lga, "summary_file", sinker, "lst.lga.@summaryfile")

        def getsubs(t1,t2):
            # Make substitutions
            subs = [("_Nt1_%d/_Nt2_%d/_segstats_lga0/summary.stats"%(i,j),
                     "{}/{}_summary.stats".format(t1[i].split("/")[-1].split(".nii.gz")[0],
                                     t2[j].split("/")[-1].split(".nii.gz")[0])) \
                     for i in range(len(t1)) for j in range(len(t2))]
            subs += [("_Nt1_%d/_Nt2_%d"%(i,j),t1[i].split("/")[-1].split(".nii.gz")[0]) for i in range(len(t1)) for j in range(len(t2))]
            subs += [("_Nt2_%d"%i, "") for i in range(10)]
            subs += [("_cluster_lga0", "")]
            return subs

        subs = Node(Function(input_names=["t1", "t2"],
                             output_names=["subs"],
                             function=getsubs),
                    name="subs")
        #sinker.inputs.substitutions = getsubs()
        wf.connect(map_func, "t1", subs, "t1")
        wf.connect(map_func2, "t2", subs, "t2")
        wf.connect(subs, "subs", sinker, "substitutions")
        wf.connect(lga, "lesion_probability_map", sinker, "lst.lga.@map")
        wf.connect(lga, "mat_file", sinker, "lst.lga.@mat")

        wf.config = {"execution": {"crashdump_dir": os.path.join(config["crash_directory"],
                                                                 self.inputs.mseID,
                                                                 self.flag)}}
        wf.run(plugin=self.inputs.plugin,
               plugin_args=self.inputs.plugin_args)

        return runtime


    def _get_output_folder(self):
        # output folder for status.json
        return "lst/lga"

    def get_metrics(self, f):
        import numpy as np
        if os.path.exists(f):
            foo = np.genfromtxt(f)
            print("foo.shape is", foo.shape)
            if len(foo.shape)==2:
                data = foo[:,3][1:]
                output = {"number_of_lesions": data.shape[0],
                          "total_lesion_volume": np.sum(data),
                          "max_lesion_size": np.max(data)}
                return output
            else:
                return {"number_of_lesions": 0}
        else:
            raise FileNotFoundError


    def _list_outputs(self):
        # managing outputs for later calculate Dice Coefficient
        import shutil  # copy2 also copy file metadata like file's creation and modification times
        import numpy as np
        from glob import glob

        bin_map_dir = "/data/henry1/tristan/LST/FLAIR-MPRAGE" # TODO make a dictionary
        bin_map = "*-{0}*_bin_lesion_map.nii".format(self.inputs.mseID)
        src = ''.join(glob(os.path.join(bin_map_dir, self.inputs.mseID, bin_map)))
        dstnames = glob(os.path.join(config["output_directory"], self.inputs.mseID,
                                     "lst", "lga", "*", "_kappa_*"))
        for dst in dstnames:
            shutil.copy(src, dst)

        outputs = self._outputs().get()

        outputs["lesion_probability_mask_lga"] = sorted(glob(os.path.join(config["output_directory"],
                                                   self.inputs.mseID, "lst","lga","*",
                                                   "_kappa_*", "ples_lga*.nii")))
        outputs["lesion_binary_ref"] = sorted(glob(os.path.join(config["output_directory"], self.inputs.mseID,
                                                                "lst", "lga", "*", "_kappa_*", bin_map)))
        outputs["lga_mat"] = sorted(glob(os.path.join(config["output_directory"],
                                                   self.inputs.mseID, "lst","lga","*",
                                                   "_kappa_*", "*.mat")))
        outputs["lga_txt"] = sorted(glob(os.path.join(config["output_directory"],
                                                   self.inputs.mseID, "lst","lga","*",
                                                   "_kappa_*", "_segstats_lga0", "*.stats")))
        outputs["lga_index"] = sorted(glob(os.path.join(config["output_directory"],
                                                   self.inputs.mseID, "lst","lga","*",
                                                   "_kappa_*", "cluster", "*_index.nii.gz")))
        outputs["lga_metrics"] = [self.get_metrics(f) for f in outputs["lga_txt"]]

        return outputs

register_workflow(LGAIter)

__PBR interface for running LGA with kappa of increment 0.05__

In [ ]:
__author__ = 'sf713420'

import os
from nipype.interfaces.base import isdefined
from utils import doit_workflow, flatten
import sys
from glob import glob
import numpy as np
import csv

if __name__ == '__main__':
    iter = sys.argv[1]
    mse_test = ['mse3727', 'mse4413', 'mse4482', 'mse4739', 'mse4754']
    data_ref = [glob('/data/henry7/PBR/subjects/{0}/lst/lga/ms*/*/*_bin_lesion_map.nii'.format(mse)) for mse in mse_test]
    data_ref = flatten(data_ref)
    print(data_ref,'\n', len(data_ref))

    output_dir = '/data/henry1/tristan/LST/opt_thresh_results/test_subjects'
    sk_dir = os.path.join(output_dir)
    
    if iter == '-iter':
        thresh_array = np.linspace(0.05, 1.00, num=20)
    elif isdefined(iter):
        raise ValueError("The option for iterate binary threshold is -iter")

    dwf = doit_workflow(data_ref, thresh_array, sink_dir=sk_dir)
    dwf.run()

__This script takes the first positional argument as option for either iterating or not binary threshold for running the LGA DC calculation workflow. __

### C. New Metric

The new metric take both voxel-based and lesion-based into account. The voxel-based DC (vDC) was already calculated in previous method. To calculate lesion-based DC (lDC), an algorithm to find centroids of lesions was implemented. The script below will get centroids of both:
1) lst_edits (reference) and map it to lga lesion. By doing this, False Negative (FN) and True Positive (TP1) are calculated.
2) LGA probability lesion mapping it back to lst_edits. By doing this, False Positive (FP) and True Positive (TP2) are calculated

As is known that vDC is caculated based on the formula:
<center>vDC = 2TP / (2TP + FP + FN)<center>

And lDC is calculated:
<center>lDC = (TP1 + TP2) / (TP1 + TP2 + FP + FN)<center>

The new metric (nDC) takes sum of 0.5 weighting of vDC and 0.5 weighting of lDC:
<center>nDC = 0.5 * vDC + 0.5 * lDC<center>  


In [ ]:
__author__ = 'sf713420'
import os
import numpy as np
import nibabel as nib
from scipy.ndimage import label
from nipype.utils.filemanip import save_json, load_json
from copy import deepcopy
import operator

class GetCenterLesion():

    def __init__(self, filename_list):
        self.filename_list = filename_list
        self.centroids = {}
        self.lesion_size = []

    def run_centroids(self):
        for filename in self.filename_list:
            if len(self.filename_list) == 1:
                kappa = 'reference'
            else:
                kappa = os.path.split(filename)[1].split('_')[2]
                if kappa == '1':
                    kappa = kappa + '.0'

            img = nib.load(filename)
            data = img.get_data()

            # print(img.header)
            labeled_img, nlabels = label(data > 0)
            self.lesion_size = np.bincount(labeled_img.ravel())
            # print(self.lesion_size)
            centroid = {}
            centroid['present'] = {}
            centroid['missing'] = {}
            for idx in range(1, len(self.lesion_size)):
                # idx is the index for labeled lesions
                # centroid['present'][idx] = {}
                l = (labeled_img == idx)
                l_coord = np.nonzero(l)
                # print(labeled_img.shape)
                # print(l_coord)

                x = x_sub = l_coord[0]
                y = y_sub = l_coord[1]
                z = z_sub = l_coord[2]
                x_mean = np.mean(x)
                y_mean = np.mean(y)
                z_mean = np.mean(z)

                x_round = int(round(x_mean))
                y_round = int(round(y_mean))
                z_round = int(round(z_mean))
                # print(x_round, y_round, z_round)

                for i in range(self.lesion_size[idx]):
                    if x_round == x[i] and y_round == y[i] and z_round == z[i]:
                        # print("Yes the centroid is in the shape")
                        # print(labeled_img[x_round, y_round, z_round])
                        centroid['present'][idx] = [x_round, y_round, z_round]
                        # centroid['present'][idx]['LesionSize'] = self.lesion_size[idx]
                        break
                    x_sub = np.delete(x_sub, 0)
                    y_sub = np.delete(y_sub, 0)
                    z_sub = np.delete(z_sub, 0)

                if  len(x_sub) == 0 and len(y_sub) == 0 and len(z_sub) == 0:
                    # centroid['missing'][idx] = [x_round, y_round, z_round]
                    # assumption is that there must be voxels in z_round plane
                    print("Oops, this centroid is not located in the lesion shape \n",
                          "The coordinate is (x, y, z): ", [x_round, y_round, z_round], '\n',
                          "lesion size is: ", self.lesion_size[idx], "\n",
                          "The lesion coordinate is: ", l_coord,
                          "\n",
                          "going to the 2nd level")

                    x_in_right = []
                    y_in_right = []
                    miss_coord_right = []
                    x_in_left = []
                    y_in_left = []
                    miss_coord_left = []
                    x_in_up = []
                    y_in_up = []
                    miss_coord_up = []
                    x_in_down = []
                    y_in_down = []
                    miss_coord_down = []
                    for i in range(self.lesion_size[idx]):
                        if z[i] == z_round:
                            if x[i] >= x_round:
                                x_in_right.append(x[i])
                                y_in_right.append(y[i])
                                miss_coord_right.append([x[i], y[i], z[i]])
                            if x[i] <= x_round:
                                x_in_left.append(x[i])
                                y_in_left.append(y[i])
                                miss_coord_left.append((x[i], y[i], z[i]))
                            if y[i] >= y_round:
                                x_in_up.append(x[i])
                                y_in_up.append(y[i])
                                miss_coord_up.append([x[i], y[i], z[i]])
                            if y[i] <= y_round:
                                x_in_down.append(x[i])
                                y_in_down.append(y[i])
                                miss_coord_down.append([x[i], y[i], z[i]])

                    # print(miss_coord)
                    #coord_list = [len(miss_coord_right), len(miss_coord_up), len(miss_coord_left), len(miss_coord_down)]
                    #max_index, max_value = max(enumerate(coord_list), key=operator.itemgetter(1))

                    if len(miss_coord_right) >= len(miss_coord_left):
                        new_miss_coord_lr = miss_coord_right
                        x_in_new_lr = x_in_right
                        y_in_new_lr = y_in_right
                    else:
                        new_miss_coord_lr = miss_coord_left
                        x_in_new_lr = x_in_left
                        y_in_new_lr = y_in_left

                    if len(miss_coord_up) >= len(miss_coord_down):
                        new_miss_coord_ud = miss_coord_up
                        x_in_new_ud = x_in_up
                        y_in_new_ud = y_in_up
                    else:
                        new_miss_coord_ud = miss_coord_down
                        x_in_new_ud = x_in_down
                        y_in_new_ud = y_in_down

                    if len(new_miss_coord_lr) >= len(new_miss_coord_ud):
                        x_in_new = x_in_new_lr
                        y_in_new = y_in_new_lr
                        new_miss_coord = new_miss_coord_lr
                    else:
                        x_in_new = x_in_new_ud
                        y_in_new = y_in_new_ud
                        new_miss_coord = new_miss_coord_ud

                    x_round2 = int(round(np.mean(x_in_new)))
                    y_round2 = int(round(np.mean(y_in_new)))
                    z_round2 = z_round
                    x_sub2 = x_in_new
                    y_sub2 = y_in_new
                    z_sub2 = [z_round2] * len(new_miss_coord)

                    for i in range(len(new_miss_coord)):
                        if x_round2 == x_in_new[i] and y_round2 == y_in_new[i]:
                            centroid['present'][idx] = [x_round2, y_round2, z_round2]
                            # centroid['present'][idx]['LesionSize'] = self.lesion_size[idx]
                            break
                        x_sub2 = np.delete(x_sub2, 0)
                        y_sub2 = np.delete(y_sub2, 0)
                        z_sub2 = np.delete(z_sub2, 0)

                    if len(x_sub2) == 0 and len(y_sub2) == 0 and len(z_sub2) == 0:
                        print("Oops, this centroid is still not in the lesion shape \n",
                              "The coordinate is (x, y, z): ", [x_round2, y_round2, z_round2], '\n',
                              "lesion size is: ", len(new_miss_coord), "\n",
                              "The lesion coordinate is: ", new_miss_coord,
                              "\n",
                              "going to the 3rd level")

                        # Assumption is that there must be voxels in y_round2 axis
                        x_in_right2D = []
                        miss_coord_right2D = []
                        x_in_left2D = []
                        miss_coord_left2D = []
                        y_in_up2D = []
                        miss_coord_up2D = []
                        y_in_down2D = []
                        miss_coord_down2D = []

                        for i in range(len(new_miss_coord)):
                            if y_in_new[i] == y_round2:
                                if x_in_new[i] >= x_round2:
                                    x_in_right2D.append(x_in_new[i])
                                    miss_coord_right2D.append([x_in_new[i], y_in_new[i]])
                                if x_in_new[i] <= x_round2:
                                    x_in_left2D.append(x_in_new[i])
                                    miss_coord_left2D.append([x_in_new[i], y_in_new[i]])
                            if x_in_new[i] == x_round2:
                                if y_in_new[i] >= y_round2:
                                    y_in_up2D.append(y_in_new[i])
                                    miss_coord_up2D.append([x_in_new[i], y_in_new[i]])
                                if y_in_new[i] <= y_round2:
                                    y_in_down2D.append(y_in_new[i])
                                    miss_coord_down2D.append([x_in_new[i], y_in_new[i]])

                        if len(miss_coord_right2D) >= len(miss_coord_left2D):
                            x_in_new2D = x_in_right2D
                            new_miss_coord2D_lr = miss_coord_right2D
                        else:
                            x_in_new2D = x_in_left2D
                            new_miss_coord2D_lr = miss_coord_left2D

                        if len(miss_coord_up2D) >= len(miss_coord_down2D):
                            y_in_new2D = y_in_up2D
                            new_miss_coord2D_ud = miss_coord_up2D
                        else:
                            y_in_new2D = y_in_down2D
                            new_miss_coord2D_ud = miss_coord_down2D

                        if len(x_in_new2D) >= len(y_in_new2D):
                            x_round3 = int(round(np.mean(x_in_new2D)))
                            y_round3 = y_round2
                            z_round3 = z_round2
                            new_miss_coord2D = new_miss_coord2D_lr
                            x_sub3 = x_in_new2D
                            y_sub3 = [y_round3] * len(new_miss_coord2D)
                            z_sub3 = [z_round3] * len(new_miss_coord2D)

                            for i in range(len(new_miss_coord2D)):
                                if x_in_new2D[i] == x_round3:
                                    centroid['present'][idx] = [x_round3, y_round3, z_round3]
                                    print("Congrats! The centroid is found in the 3rd level")
                                    # centroid['present'][idx]['LesionSize'] = self.lesion_size[idx]
                                    break
                                x_sub3 = np.delete(x_sub3, 0)
                                y_sub3 = np.delete(y_sub3, 0)
                                z_sub3 = np.delete(z_sub3, 0)
                        else:
                            x_round3 = x_round2
                            y_round3 = int(round(np.mean(y_in_new2D)))
                            z_round3 = z_round2
                            new_miss_coord2D = new_miss_coord2D_ud
                            x_sub3 = [x_round3] * len(new_miss_coord2D)
                            y_sub3 = y_in_new2D
                            z_sub3 = [z_round3] * len(new_miss_coord2D)

                            for i in range(len(new_miss_coord2D)):
                                if y_in_new2D[i] == y_round3:
                                    centroid['present'][idx] = [x_round3, y_round3, z_round3]
                                    print("Congrats! The centroid is found in the 3rd level")
                                    # centroid['present'][idx]['LesionSize'] = self.lesion_size[idx]
                                    break
                                x_sub3 = np.delete(x_sub3, 0)
                                y_sub3 = np.delete(y_sub3, 0)
                                z_sub3 = np.delete(z_sub3, 0)

                        if len(x_sub3) == 0 and len(y_sub3) == 0 and len(z_sub3) == 0:
                            # centroid['missing'][idx] = {}
                            centroid['missing'][idx] = {'Level1': [x_round, y_round, z_round],
                                                               'Level2': [x_round2, y_round2, z_round2],
                                                               'Level3': [x_round3, y_round3, z_round3]}
                            # centroid['missing'][idx]['LesionSize'] = self.lesion_size[idx]
                            print("Oops, this algorithm did not work. \n",
                                  "The coordinate is (x, y, z): ", [x_round3, y_round3, z_round3], '\n',
                                  "lesion size is: ", self.len(new_miss_coord2D), "\n",
                                  "The lesion coordinate is: ", new_miss_coord2D,
                                  "\n",
                                  "All three levels of centroids are stored in the json file.")

            if len(centroid['present']) == idx:
                print("Great, all the centroids are in the lesion shape! The kappa is: ", kappa)
            elif not centroid['present'] and not centroid['missing']:
                print("There was no lesion found in this scenario. The kappa is: ", kappa)
            else:
                raise ValueError(":( Please try other algorithm to get a valid point of the lesion shape")
            if kappa == 'reference':
                kappa_name = kappa
            else:
                kappa_name = '_kappa_' + kappa

            centroid_temp = deepcopy(centroid)
            self.centroids[kappa_name] = centroid_temp

        return self.centroids

    def make_lga_json(self, centroids = None):
        if centroids is None:
            centroids = self.run_centroids()

        outdir = os.path.split(os.path.split(self.filename_list[0])[0])[0]
        save_json(os.path.join(outdir, "centroid_lga.json"), centroids)
        print("The json file is generated in the directory: ", outdir)

def load_data(lstpath):
    img = nib.load(lstpath)
    data = img.get_data()
    return data

def cal_FP(f, lst_edit):
    kappa_array = np.linspace(0.05, 1.0, 20)
    if len(lst_edit) > 1:
        raise ValueError("More than one lst_edits files, please check")
    elif len(lst_edit) == 0:
        raise ValueError("No lst_edits file is found, please check your folder")
    elif len(lst_edit) == 1:
        lst_edit = ''.join(lst_edit)
    print("The lst_edit is in the directory: ", lst_edit)
    lst_data = load_data(lst_edit)
    # print(np.max(lst_data), np.min(lst_data))
    Lesion = GetCenterLesion(f)
    lesions = Lesion.run_centroids()
    for num in kappa_array:
        num_str = str(num)
        kappa_name = '_kappa_' + num_str
        if len(lesions[kappa_name]['missing']) != 0:
            raise ValueError("There is some centroid missed from the algorithm. "
                             "Please make sure all the center points are found in the lesion.")
        else:
            print("Awesome, all the centroids are present in the lesion.")

        lesion_items = sorted(lesions[kappa_name]['present'].items())
        FP = 0
        TP = 0
        lesions[kappa_name]['FalsePositives'] = []
        lesions[kappa_name]['TruePositives_to_ref'] = []
        for i, coord in lesion_items:
            if lst_data[coord[0], coord[1], coord[2]] == 0:
            # if lst_data[coord['xyz'][0], coord['xyz'][1], coord['xyz'][2]] == 0:
                lesions[kappa_name]['FalsePositives'].append(i)
                FP += 1
            else:
                lesions[kappa_name]['TruePositives_to_ref'].append(i)
                TP += 1

        lesions[kappa_name]['NumOfFP'] = FP
        lesions[kappa_name]['NumOfTP_to_ref'] = TP
    Lesion.make_lga_json(lesions)
    return [FP, TP]

def cal_FN(f, lst_edit):
    kappa_array = np.linspace(0.05, 1.0, 20)
    if len(lst_edit) > 1:
        raise ValueError("More than one lst_edits files, please check")
    elif len(lst_edit) == 0:
        raise ValueError("No lst_edits file is found, please check your folder")

    print("The lst_edit is in the directory: ", lst_edit)
    lga_data = [load_data(f_kappa) for f_kappa in f]
    print("There are {} elements in lga data".format(len(lga_data)))
    # print("Lga 0 is: ", lga_data[0])
    print("Check if it's 1 for mse3727: ", lga_data[0][52,102,47], lga_data[0][52,103,47])

    Lesion = GetCenterLesion(lst_edit)
    lesions = Lesion.run_centroids()
    ref_lesion = 'reference'
    if len(lesions[ref_lesion]['missing']) != 0:
        raise ValueError("There is some centroid missed from the algorithm. "
                             "Please make sure all the center points are found in the lesion.")
    else:
        print("Awesome, all the centroids are present in the lesion.")

    ref_lesion_to_kappas = {}

    for k, num in enumerate(kappa_array):
        num_str = str(num)
        kappa_name = '_kappa_' + num_str
        lesion_items = sorted(lesions[ref_lesion]['present'].items())
        print("Lesion items are: ", lesion_items)
        FN = 0
        TP = 0
        lesions[ref_lesion]['FalseNegatives'] = []
        lesions[ref_lesion]['TruePositives_to_lga'] = []
        for i, coord in lesion_items:
            print(i, coord)
            if lga_data[k][coord[0], coord[1], coord[2]] == 0:
            # if lga_data[coord['xyz'][0], coord['xyz'][1], coord['xyz'][2]] == 0:
                lesions[ref_lesion]['FalseNegatives'].append(i)
                FN += 1
                print("Oops, FN plus 1! FN is: ", FN)
            else:
                lesions[ref_lesion]['TruePositives_to_lga'].append(i)
                TP += 1
                print("Yay! TP plus 1! TP is: ", TP)

        print("FN, TP and kappa are: ", FN, TP, kappa_name)
        lesions[ref_lesion]['NumOfFN'] = FN
        lesions[ref_lesion]['NumOfTP_to_lga'] = TP
        print("After adding FN and TP, the lesion dictionary is: ", lesions)
        lesions_temp = deepcopy(lesions)
        ref_lesion_to_kappas[kappa_name] = lesions_temp
        print("After making a bigger dict to ref_lesion_to_kappas: ", ref_lesion_to_kappas)

    print("After all kappas, the ref_lesion_to_kappas looks: ", ref_lesion_to_kappas)
    outdir = os.path.split(os.path.split(f[0])[0])[0]
    save_json(os.path.join(outdir, "centroid_lst_edit.json"), ref_lesion_to_kappas)
    print("The json file is generated in the directory: ", outdir)
    return [FN, TP]


if __name__ == '__main__':
    from glob import glob
    test_mse = ['mse3727', 'mse4413', 'mse4482', 'mse4739', 'mse4754']
    FPTP = {}
    FNTP = {}
    for mse in test_mse:
        f = sorted(glob('/data/henry7/PBR/subjects/{0}/lst/lga/ms*/_kappa_*/ples_lga_*_rmms*.nii'.format(mse)))
        lst_edit = glob('/data/henry7/PBR/subjects/{0}/mindcontrol/ms*/lst/lst_edits/no_FP_filled_FN*'.format(mse))
        FPTP[mse] = cal_FP(f, lst_edit)
        # for cal_FN debugging
        FNTP[mse] = cal_FN(f, lst_edit)
        # A lot 2nd level centroid in mse4482 and mse4754
        # No lesion for mse3327 when kappa is 1.0
        # No lesion for mse4439 when kappa is or greater than 0.8

    print(FPTP, FNTP)
    """
        outdir = os.path.split(os.path.split(f[0])[0])[0]
        save_json(os.path.join(outdir, "centroid_lga.json"), lesions)
    """


## III. Results

### A. Table of Binary Threshold Results

In [1]:
# from IPython.display import display, HTML
import pandas as pd
df1 = pd.read_csv("/data/henry1/tristan/LST/opt_thresh_results/FLAIR-MPRAGE/averages.csv")
df1

,Threshold,Dice_Coefficient,SE,SP
0,0.05,0.475833,0.358878,0.999789
1,0.10,0.468314,0.349578,0.999817
2,0.15,0.463385,0.343767,0.999832
3,0.20,0.459555,0.339401,0.999842
4,0.25,0.456596,0.336025,0.999849
5,0.30,0.454107,0.333195,0.999855
6,0.35,0.451778,0.330667,0.999859
7,0.40,0.449665,0.328433,0.999862
8,0.45,0.447638,0.326332,0.999865
9,0.50,0.446124,0.324714,0.999868


Table 1. Averaged Dice Coefficient, SE and SP for all FLAIR-MPRAGE mses with 0.05 increment of binary threshold.

As is shown in the table, DC decreases with binary threshold increasing. This indicate that the binary threshold has monotonous effect in determining DC. In this case, we will use binary threshold = 0.3 for the later determination of optimal initial threshold.

### B. Table of Initial Threshold Results

In [3]:
import pandas as pd
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')
df2 = pd.read_csv("/data/henry1/tristan/LST/opt_thresh_results/test_subjects/_bin_thresh_0.3/For_Jupyter_Display.csv")
print_full(df2)

                         FLAIR  kappa        DC        SE       SP
0   rmms1624-mse3727-077-FLAIR   0.05  0.551630  0.481400  0.99923
1   rmms1624-mse3727-077-FLAIR   0.10  0.533100  0.379960  0.99987
2   rmms1624-mse3727-077-FLAIR   0.15  0.462580  0.304510  0.99996
3   rmms1624-mse3727-077-FLAIR   0.20  0.395480  0.247350  0.99999
4   rmms1624-mse3727-077-FLAIR   0.25  0.338160  0.203700  1.00000
5   rmms1624-mse3727-077-FLAIR   0.30  0.312060  0.184990  1.00000
6   rmms1624-mse3727-077-FLAIR   0.35  0.269330  0.155680  1.00000
7   rmms1624-mse3727-077-FLAIR   0.40  0.248320  0.141760  1.00000
8   rmms1624-mse3727-077-FLAIR   0.45  0.233200  0.131990  1.00000
9   rmms1624-mse3727-077-FLAIR   0.50  0.206520  0.115150  1.00000
10  rmms1624-mse3727-077-FLAIR   0.55  0.196780  0.109120  1.00000
11  rmms1624-mse3727-077-FLAIR   0.60  0.192030  0.106210  1.00000
12  rmms1624-mse3727-077-FLAIR   0.65  0.178690  0.098109  1.00000
13  rmms1624-mse3727-077-FLAIR   0.70  0.164090  0.089379  1.0

Table 2. Initial threshold (kappa) with increment of 0.05 for 5 mse subjects. The DC, SE and SP are displayed.

### C. New Metric (To be continued)



## IV. Discussion

### A. 
The threshold in Table 1 is binary threshold for the computation of LGA binary lesion maps, The threshold can be set between 0 to 1, where the thresholding is more liberal when it's set close to 0 and more conservative when it's set close 1. 
We concluded from Table 1 the binary threshold has a monotonous effect in determining DC, it will not affect the accuracy of determination of kappa across different subjects as long as binary threshold is kept the same. In this case, binary threshold is set to 0.3 for future determination of optimal initial threshold.

### B. 
The table in our results displays the outputs of LST’s provided tool for determining the optimal threshold for 5 test subjects iterated over 20 values of kappa (0.05 - 1.0).  Our iteration through various kappa values shows us that the Dice Coefficient is generally inversely proportional to the initial threshold. This implies that a lower kappa value will produce a more accurate lesion map. However, upon inspection of the lesion maps in comparison to manually created and edited lesion maps of the same test subject it appeared that this was not necessarily the case. It is true that we see the least False Negatives in the lesion maps created with the lowest kappa values. However these maps also create False Positives due to their extremely low threshold, and in some cases did not appear to have the most accurate lesion maps. This disparity between the lesion map with the highest Dice Coefficient and the most accurate lesion map calls for a new metric to be used in measuring LGA lesion maps’ accuracy. 